# System Model V3 - Debt Market Model - Ideal Parameters

The purpose of this notebook is to simulate a single run of the full CDP and APT system model, using a stochastic Ethereum price and liquidity demand process as a driver.


We want to run a basic simuluation in order to serve as a sanity check. We want the simulation to have no liquidity shocks, the controller off, arbitrageur considers the liquidation ratio, and target price scaling. The market price presented should be stable and should reflect the movement and volatility of the realized sample path of the ETH price

In order to test this, configure the following experiment [ideal_params.py](https://github.com/BlockScience/reflexer/blob/experiment-analysis/experiments/system_model_v3/ideal_params.py). Run run this simulation, we create a directory in the ```experiments/system_model_v3``` called ```ideal_params/```,and add a ```logs/``` directory inside of it.

Assuming our we have all of the requirements required (run requirements.txt from the ```reflexer/``` root directory to be sure. Assuming our terminal is in the root directory, we run the follow to run the simulation:

```bash
cd experiments/system_model_v3
python3 -m experiments.system_model_v3.ideal_params
```
And our simulation will run. The resulting [run log](https://github.com/BlockScience/reflexer/blob/experiment-analysis/experiments/system_model_v3/ideal_params/experiment_run_log.md)
 can be found in the ```experiments/system_model_v3/ideal_params/``` directory, along with the simulation results stored as ```experiment_results.hdf5```


Below we will import and examine the simulation results.


In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

## Import simulation run data


In [ ]:
os.getcwd()

In [ ]:
os.chdir('../../')
os.getcwd()

In [ ]:
experiment_results = 'experiments/system_model_v3/unstable_controller_params/experiment_results.hdf5'
experiment_results_keys = []
with pd.HDFStore(experiment_results) as store:
    experiment_results_keys = list(filter(lambda x: "results" in x, store.keys()))
    exceptions_keys = list(filter(lambda x: "exceptions" in x, store.keys()))

In [ ]:
# A list of all experiment result keys
experiment_results_keys

In [ ]:
# Copy a results_ key from the above keys to select the experiment
experiment_results_key = experiment_results_keys[-1]#'results_2021-02-09T18:46:33.073363' # Or select last result: experiment_results_keys[-1]
experiment_timestamp = experiment_results_key.strip('results_')
exceptions_key = 'exceptions_' + experiment_timestamp
experiment_timestamp

In [ ]:
df_raw = pd.read_hdf(experiment_results, experiment_results_key)
df_raw.tail()

## Post process

In [ ]:
from experiments.system_model_v3.post_process import post_process_results
from experiments.system_model_v3.unstable_controller_params import params, SIMULATION_TIMESTEPS

In [ ]:
df = post_process_results(df_raw, params, set_params=['ki', 'kp', 'liquidation_ratio'])
df

# Simulation Analysis

In [ ]:
df.substep.unique()

In [ ]:
df.subset.unique()

In [ ]:
df.columns

### Generate key plots for analysis

In [ ]:
sns.lineplot(data=df,x="timestamp", y="eth_price",label='Generated Eth price')
plt.ylim(250, 600)
ax2 = plt.twinx()
sns.lineplot(data=df,x="timestamp", y="market_price",ax=ax2,color='r',label='Market Price in Rai')
sns.lineplot(data=df,x="timestamp", y="target_price_scaled",ax=ax2,color='g',label='Redemption Price in Rai')
plt.title('Generated Eth price vs Simulation Market and Redemption Prices')
plt.legend(loc="upper left")
plt.savefig('experiments/system_model_v3/unstable_controller_params/ideal_params.png')

In [ ]:
sns.lineplot(data=df,x="timestamp", y="RAI_balance",color='r',label='Rai Balance in Rai')
sns.lineplot(data=df,x="timestamp", y="principal_debt",color='g',label='CDP total debt (RAI)')
plt.title('RAI balances in CDP/Uniswap')
plt.legend(loc="upper left")
plt.savefig('experiments/system_model_v3/unstable_controller_params/RAI_balances_in_CDPUniswap.png')

In [ ]:
sns.lineplot(data=df,x="timestamp", y="eth_locked",label='Total Eth locked')
plt.legend(loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df,x="timestamp", y="target_rate",label='Redemption Rate')
plt.legend(loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df,x="timestamp", y="ETH_balance",label='Eth Balance')
plt.legend(loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df,x="timestamp", y="RAI_balance",label='Rai Balance')
plt.legend(loc="upper left")
plt.show()

## Conclusion